In [1]:
# EEE4120F - YODA Project - JEDI
# Imports
from PIL import Image
import wave
from scipy.io import wavfile
import time

def encoding(audio,png, LSB_encoding_depth):
    #---------------------------------------------------------------------------------------------------------------------------------------- 
    # Input: audio contains the wav file
    # Input: png contains the image
    # Input: LSB_encoding_depth defines the amount of bits that should be encoded in each RGB colour code
    # Output: A png image that is encoded with the secret message
    #
    # This function will take in an audio (wav) and image (png) and will encode the audio message within the image data using stenography. 
    #------------------------------------------------------------------------------------------------------------------------------------------

    # Open the wav file
    wav_file = wave.open(audio) 
    
    # Get the file properties
    num_channels = wav_file.getnchannels()
    sample_width = wav_file.getsampwidth()
    sample_rate = wav_file.getframerate()
    num_frames = wav_file.getnframes()
    duration = num_frames / sample_rate  

    # Close the file
    wav_file.close()

    # Open the wav file using wavfile
    samplerate, data = wavfile.read(audio)
    
    # Convert audio to single channel
    if num_channels != 1:
        data = data[:, 0]
    
    # Shift all data samples up by 2^15 to make all samples positive
    data = data + 2**15
    
    # Convert data to a bitstream 
    audio_bitstream = ""
    for sample in data:
        audio_bitstream += format(sample, '016b')

    # Saves the encoded bit stream to a file
    with open("OriginalAudioBitstream.txt", "w") as file:
        for i in range(0, len(audio_bitstream), 8):
            byte = audio_bitstream[i:i+8]  # Extracting 8 bits (1 byte)
            file.write(byte + "\n")
    
    # Get length of audio_bitstream
    audio_length = len(audio_bitstream)
    audio_length_binary = format(audio_length, '024b')
    
    # Attach audio length to the audio bitstream
    a_bitstream = audio_bitstream
    audio_bitstream = audio_length_binary + audio_bitstream

    
    # Begin Image processing 
    # Load the PNG image
    image = Image.open(png)
    
    # Get the size of the image
    width, height = image.size
    
    # Checks image to ensure correct size
    if width != 1000 or height != 1000:       
        print("Wrong image size!")
        print("Please try again.")
        exit()
    else:
        # Convert the image to RGB mode 
        image_rgb = image.convert("RGB")
    
        # Get pixel data of image
        pixels = image_rgb.load()
    
        # Convert RGB values to binary and store in a 2D array
        binary_pixels = []
        for y in range(image.height):
            row = []
            for x in range(image.width):
                r, g, b = pixels[x, y]
                binary_r = bin(r)[2:].zfill(8)  # Convert red value to binary with 8 bits
                binary_g = bin(g)[2:].zfill(8)  # Convert green value to binary with 8 bits
                binary_b = bin(b)[2:].zfill(8)  # Convert blue value to binary with 8 bits
                row.append((binary_r, binary_g, binary_b))
            binary_pixels.append(row)

    
    # Begin encoding process 
    encoded_golden_measure = Image.new("RGB", (width, height))
    # Convert binary pixel data back to image and encode the image simultaneously
    encoded_pixels = encoded_golden_measure.load()
    
    # Initialise variables to store the continuous bit stream
    encoded_bitstream = "" # Will contain the encoded image bitstream for sending
    stream_index = 0

    # Iterates through each pixel in the image
    for y in range(image.height):
        row = []
        for x in range(image.width):
            # Get the RGB values of the current pixel
            binary_r, binary_g, binary_b = binary_pixels[y][x]
            # Modifies last two digits of each color component if audio stream has bits left
            if stream_index < len(audio_bitstream) - LSB_encoding_depth:
                r = binary_r[:-LSB_encoding_depth]
                for n in range(LSB_encoding_depth):
                    r += audio_bitstream[stream_index + n]
                stream_index += LSB_encoding_depth
            else:
                r = binary_r
                
            if stream_index < len(audio_bitstream) -  LSB_encoding_depth:
                g = binary_g[:-LSB_encoding_depth]
                for n in range(LSB_encoding_depth):
                    g += audio_bitstream[stream_index + n]
                stream_index += LSB_encoding_depth
            else:
                g = binary_g
                
            if stream_index < len(audio_bitstream) -  LSB_encoding_depth:
                b = binary_b[:-LSB_encoding_depth]
                for n in range(LSB_encoding_depth):
                    b += audio_bitstream[stream_index + n]
                stream_index += LSB_encoding_depth
            else:
                b = binary_b
    
            # Store modified RGB values in the encoded array
            row.append((r, g, b))
    
            # Append the modified bits to the continuous bit stream
            encoded_bitstream += r + g + b
    
            # Convert binary RGB values back to integer
            r_int = int(r, 2)
            g_int = int(g, 2)
            b_int = int(b, 2)
            
            # Set pixel value in the new image
            encoded_pixels[x, y] = (r_int, g_int, b_int)

    # Saves the encoded bit stream to a file
    with open("EncodedImageBitstream.txt", "w") as file:
        for i in range(0, len(encoded_bitstream), 8):
            byte = encoded_bitstream[i:i+8]  # Extracting 8 bits (1 byte)
            file.write(byte + "\n")

    # Save the encoded image
    encoded_golden_measure.save("Encoded_Image.png")  
    return encoded_bitstream

In [8]:
start_time = time.time()

a = "./obiwan.wav"
i = "./babyyoda.png"
b = encoding(a, i, 2)

# Calculate the elapsed time
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Elapsed time: {elapsed_time} seconds")



3096576
Elapsed time: 54.87256407737732 seconds


In [5]:
from PIL import Image
import wave
from scipy.io import wavfile
import time
import numpy as np

def decoding(encoded_image, LSB_encoding_depth):
    # Load the PNG image
    image = Image.open(encoded_image)
    
    # Get the size of the image
    width, height = image.size

    # Convert the image to RGB mode 
    image_rgb = image.convert("RGB")

    # Get pixel data of image
    pixels = image_rgb.load()

    # Convert RGB values to binary and store in a 2D array
    image_bitstream = ''
    for y in range(image.height):
        for x in range(image.width):
            r, g, b = pixels[x, y]
            binary_r = bin(r)[2:].zfill(8)  # Convert red value to binary with 8 bits
            binary_g = bin(g)[2:].zfill(8)  # Convert green value to binary with 8 bits
            binary_b = bin(b)[2:].zfill(8)  # Convert blue value to binary with 8 bits
            image_bitstream += binary_r + binary_g + binary_b


    # time from here
    start_time1 = time.time()
    #
    # # Find length of audio by extracting the first 24 bits
    audio_length_bitstream = ''
    iterations = (24//LSB_encoding_depth)
    remainder = (24%LSB_encoding_depth)
    start = 8
    for n in range(iterations):
        audio_length_bitstream += image_bitstream[start - LSB_encoding_depth : start]
        start = start + 8
    if remainder > 0:
        audio_length_bitstream += image_bitstream[ start - LSB_encoding_depth : start - (LSB_encoding_depth - remainder)]

    #print(audio_length_bitstream)

    # Convert audio length from binary to decimal 
    audio_length = int(audio_length_bitstream, 2)

    # Cycle throught the rest of the image and extract the secret audio
    decoded_audio_bitstream = ''
    iterations = ((audio_length + 24)//LSB_encoding_depth)
    remainder = ((audio_length + 24) %LSB_encoding_depth)
    start = 8
    for n in range(iterations):
        decoded_audio_bitstream += image_bitstream[start - LSB_encoding_depth : start]
        start = start + 8
    if remainder > 0:
        decoded_audio_bitstream += image_bitstream[ start - LSB_encoding_depth : start - (LSB_encoding_depth - remainder)]
        
    #Remove audio length
    decoded_audio_bitstream = decoded_audio_bitstream[24:]


    #Convert audio back to wav file
    decoded_audio = []
    for i in range(0, len(decoded_audio_bitstream), 16):
        sample = decoded_audio_bitstream[i:i+16]
        decoded_audio.append(int(sample, 2) - 2**15)
    decoded_audio_data = np.array(decoded_audio, dtype = np.int16)
    output_file = "decoded_audio.wav"  # Specify the output file name
    wavfile.write(output_file, 44100, decoded_audio_data)

    # Calculate the elapsed time
    end_time1 = time.time()
    elapsed_time = end_time1 - start_time1

    print(f"Elapsed time: {elapsed_time} seconds")
    
    return decoded_audio_bitstream

        
 
    

In [7]:
d = decoding("Encoded_Image.png", 2)
print(d[:100])

# Saves the encoded bit stream to a file
with open("DecodedAudioBitstream.txt", "w") as file:
    for i in range(0, len(d), 8):
        byte = d[i:i+8]  # Extracting 8 bits (1 byte)
        file.write(byte + "\n")

Elapsed time: 1.6707918643951416 seconds
1000000000000000100000000000000010000000000000001000000000000000100000000000000010000000000000001000
